# 5. Health Outcomes with Linear Regression

In [26]:
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import math, scipy, numpy as np
from scipy import linalg

## Diabetes Dataset

We will use a dataset from patients with diabates.  The data consists of 442 samples and 10 variables (all are physiological characteristics), so it is tall and skinny.  The dependent variable is a quantitative measure of disease progression one year after baseline.

This is a classic dataset, famously used by Efron, Hastie, Johnstone, and Tibshirani in their [Least Angle Regression](https://arxiv.org/pdf/math/0406456.pdf) paper, and one of the [many datasets included with scikit-learn](http://scikit-learn.org/stable/datasets/).

In [27]:
data = datasets.load_diabetes()

In [28]:
feature_names=['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [29]:
trn,test,y_trn,y_test = train_test_split(data.data, data.target, test_size=0.2)

In [30]:
trn.shape, test.shape

((353, 10), (89, 10))

## Linear regression in Scikit Learn

Consider a system $Ax = b$, where $A$ has more rows than columns.  This occurs when you have more data samples than variables.  We want to find $\hat{x}$ that minimizes: 
$$ \big\vert\big\vert Ax - b \big\vert\big\vert_2$$

Let's start by using the sklearn implementation:

In [6]:
regr = linear_model.LinearRegression()
%timeit regr.fit(trn, y_trn)

535 µs ± 36.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
pred = regr.predict(test)

It will be helpful to have some metrics on how good our prediciton is.  We will look at the mean squared norm (L2), mean absolute error (L1), and $r^2$ score.

($r^2$ score is the proportion of the variance in y that is predictable from x.  For linear regression, $r^2$ is the square of the correlation coefficient between the observed outcomes and the observed predictor values.)

In [8]:
def regr_metrics(act, pred):
    return (math.sqrt(metrics.mean_squared_error(act, pred)), 
     metrics.mean_absolute_error(act, pred), 
     metrics.r2_score(act, pred))

In [9]:
regr_metrics(y_test, regr.predict(test))

(57.8656618901862, 46.199414263765327, 0.32913473080375077)

## Polynomial Features

Now, we want to try improving our model's performance by adding some more features.  Currently, our model is linear in each variable, but we can add polynomial features to change this.

In [10]:
poly = PolynomialFeatures(include_bias=False)

In [62]:
trn_feat = poly.fit_transform(trn)

In [63]:
', '.join(poly.get_feature_names(feature_names))

'age, sex, bmi, bp, s1, s2, s3, s4, s5, s6, age^2, age sex, age bmi, age bp, age s1, age s2, age s3, age s4, age s5, age s6, sex^2, sex bmi, sex bp, sex s1, sex s2, sex s3, sex s4, sex s5, sex s6, bmi^2, bmi bp, bmi s1, bmi s2, bmi s3, bmi s4, bmi s5, bmi s6, bp^2, bp s1, bp s2, bp s3, bp s4, bp s5, bp s6, s1^2, s1 s2, s1 s3, s1 s4, s1 s5, s1 s6, s2^2, s2 s3, s2 s4, s2 s5, s2 s6, s3^2, s3 s4, s3 s5, s3 s6, s4^2, s4 s5, s4 s6, s5^2, s5 s6, s6^2'

In [64]:
trn_feat.shape

(353, 65)

In [65]:
regr.fit(trn_feat, y_trn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [66]:
regr_metrics(y_test, regr.predict(poly.fit_transform(test)))

(65.40483317272528, 48.685913386795825, 0.31503822139472926)

Time is squared in #features and linear in #points, so this will get very slow!

In [67]:
%timeit poly.fit_transform(trn)

604 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Speeding up feature generation

We would like to speed this up.  We will use [Numba](http://numba.pydata.org/numba-doc/0.12.2/tutorial_firststeps.html), a Python library that compiles code directly to C.

#### Resources

[This tutorial](https://jakevdp.github.io/blog/2012/08/24/numba-vs-cython/) from Jake VanderPlas is a nice introduction and answers the question "Isn't python pretty slow?"  Here Jake [implements a non-trivial algorithm](https://jakevdp.github.io/blog/2015/02/24/optimizing-python-with-numpy-and-numba/) (non-uniform fast Fourier transform) with Numba.

<img src="images/why_python.png" alt="" style="width: 100%"/>
(source: [Jake VanderPlas](https://jakevdp.github.io/blog/2015/02/24/optimizing-python-with-numpy-and-numba/))

Here is a [thorough answer](https://softwareengineering.stackexchange.com/questions/246094/understanding-the-differences-traditional-interpreter-jit-compiler-jit-interp) on the differences between an Ahead Of Time (AOT) compiler, a Just In Time (JIT) compiler, and an interpreter.

### Experiments with vectorization and native code

Let's first get aquainted with Numba, and then we will return to our problem of polynomial features for regression on the diabates data set.

In [11]:
%matplotlib inline

In [12]:
import math, numpy as np, matplotlib.pyplot as plt
from pandas_summary import DataFrameSummary
from scipy import ndimage

In [13]:
from numba import jit, vectorize, guvectorize, cuda, float32, void, float64

We will show the impact of:
- Avoiding memory allocations and copies (slower than CPU calculations)
- Better locality
- Vectorization

If we use numpy on whole arrays at a time, it creates lots of temporaries, and can't use cache. If we use numba looping through an array item at a time, then we don't have to allocate large temporary arrays, and can reuse cached data since we're doing multiple calculations on each array item.

In [95]:
# Untype and Unvectorized
def proc_python(xx,yy):
    zz = np.zeros(nobs, dtype='float32')
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [97]:
nobs = 10000
x = np.random.randn(nobs).astype('float32')
y = np.random.randn(nobs).astype('float32')

In [99]:
%timeit proc_python(x,y)   # Untyped and unvectorized

49.6 ms ± 798 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Numpy

Numpy lets us vectorize this:

In [96]:
# Typed and Vectorized
def proc_numpy(x,y):
    z = np.zeros(nobs, dtype='float32')
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    z = z * ( z - .88 ) 
    return z

In [98]:
np.allclose( proc_numpy(x,y), proc_python(x,y), atol=1e-4 )

True

In [100]:
%timeit proc_numpy(x,y)    # Typed and vectorized

34.8 µs ± 86.1 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


#### Numba

Now we will use Numba's jit  (just-in-time) compiler decorator, without vectorizing.  This avoids large memory allocations, so we have better locality:

In [104]:
@jit()
def proc_numba(xx,yy,zz):
    for j in range(nobs):   
        x, y = xx[j], yy[j] 
        x = x*2 - ( y * 55 )
        y = x + y*2         
        z = x + y + 99      
        z = z * ( z - .88 ) 
        zz[j] = z           
    return zz

In [107]:
z = np.zeros(nobs).astype('float32')
np.allclose( proc_numpy(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [108]:
%timeit proc_numba(x,y,z)

6.21 µs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now we will use Numba's `vectorize` decorator. Simpler programming model, easier to optimize:

In [101]:
@vectorize
def vec_numba(x,y):
    x = x*2 - ( y * 55 )
    y = x + y*2         
    z = x + y + 99      
    return z * ( z - .88 ) 

In [110]:
np.allclose(vec_numba(x,y), proc_numba(x,y,z), atol=1e-4 )

True

In [111]:
%timeit vec_numba(x,y)

5.6 µs ± 37.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### Numba polynomial features

In [88]:
@jit(nopython=True)
def vec_poly(x, res):
    m,n=x.shape
    feat_idx=0
    for i in range(n):
        v1=x[:,i]
        for k in range(m): res[k,feat_idx] = v1[k]
        feat_idx+=1
        for j in range(i,n):
            for k in range(m): res[k,feat_idx] = v1[k]*x[k,j]
            feat_idx+=1

#### Row-Major vs Column-Major Storage

From this [blog post by Eli Bendersky](http://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays/):

"The row-major layout of a matrix puts the first row in contiguous memory, then the second row right after it, then the third, and so on. Column-major layout puts the first column in contiguous memory, then the second, etc.... While knowing which layout a particular data set is using is critical for good performance, there's no single answer to the question which layout 'is better' in general.

"It turns out that matching the way your algorithm works with the data layout can make or break the performance of an application.

"The short takeaway is: **always traverse the data in the order it was laid out**."

**Column-major layout**: Fortran, Matlab, R, and Julia

**Row-major layout**: C, C++, Python, Pascal, Mathematica

In [112]:
trn = np.asfortranarray(trn)
test = np.asfortranarray(test)

In [113]:
m,n=trn.shape
n_feat = n*(n+1)//2 + n
trn_feat = np.zeros((m,n_feat), order='F')
test_feat = np.zeros((len(y_test), n_feat), order='F')

In [114]:
vec_poly(trn, trn_feat)
vec_poly(test, test_feat)

In [115]:
regr.fit(trn_feat, y_trn)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [116]:
regr_metrics(y_test, regr.predict(test_feat))

(55.74734592292935, 42.836164292252306, 0.43881113706671915)

In [117]:
%timeit vec_poly(trn, trn_feat)

7.08 µs ± 21.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [118]:
605/7.7

78.57142857142857

## Regularization and noise

Regularization is a way to reduce over-fitting and create models that better generalize to new data.

### Regularization

Lasso regression uses an L1 penalty, which pushes towards sparse coefficients. The parameter $\alpha$ is used to weight the penalty term. Scikit Learn's LassoCV performs cross validation with a number of different values for $\alpha$.

Watch this [Coursera video on Lasso regression](https://www.coursera.org/learn/machine-learning-data-analysis/lecture/0KIy7/what-is-lasso-regression) for more info.

In [119]:
reg_regr = linear_model.LassoCV(n_alphas=10)

In [120]:
reg_regr.fit(trn_feat, y_trn)

/home/jhoward/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=10, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [121]:
reg_regr.alpha_

0.0098199431661591518

In [122]:
regr_metrics(y_test, reg_regr.predict(test_feat))

(50.0982471642817, 40.065199085003101, 0.54678349851873365)

### Noise

Now we will add some noise to the data

In [123]:
idxs = np.random.randint(0, len(trn), 10)

In [124]:
y_trn2 = np.copy(y_trn)
y_trn2[idxs] *= 10 # label noise

In [125]:
regr = linear_model.LinearRegression()
regr.fit(trn, y_trn)
regr_metrics(y_test, regr.predict(test))

(51.1766253181518, 41.415992803872754, 0.52706229394681392)

In [126]:
regr.fit(trn, y_trn2)
regr_metrics(y_test, regr.predict(test))

(75.36166834955054, 60.629082113104403, -0.025561385577375528)

In [127]:
hregr = linear_model.HuberRegressor()
hregr.fit(trn, y_trn2)
regr_metrics(y_test, hregr.predict(test))

(51.23666033739956, 41.858896653600304, 0.52595204184079014)

How is sklearn doing this?  By checking [the source code](https://github.com/scikit-learn/scikit-learn/blob/14031f6/sklearn/linear_model/base.py#L417), you can see that in the dense case, it calls [scipy.linalg.lstqr](https://github.com/scipy/scipy/blob/v0.19.0/scipy/linalg/basic.py#L892-L1058), which is calling a LAPACK method:

        Options are ``'gelsd'``, ``'gelsy'``, ``'gelss'``. Default
        (``'gelsd'``) is a good choice.  However, ``'gelsy'`` can be slightly
        faster on many problems.  ``'gelss'`` was used historically.  It is
        generally slow but uses less memory.

- [gelsd](https://software.intel.com/sites/products/documentation/doclib/mkl_sa/11/mkl_lapack_examples/_gelsd.htm): uses SVD and a divide-and-conquer method
- [gelsy](https://software.intel.com/en-us/node/521113): uses QR factorization
- [gelss](https://software.intel.com/en-us/node/521114): uses SVD

#### Scipy Sparse LSQR

Uses [LSQR: An Algorithm for Sparse Linear Equations and Sparse Least Squares](https://web.stanford.edu/class/cme324/paige-saunders2.pdf) by C.C. Paige and M.A. Saunders (1982).  Based on Golub and Kahan's bidiagonalization procedure.

from [scipy sparse lsqr source code](https://github.com/scipy/scipy/blob/v0.14.0/scipy/sparse/linalg/isolve/lsqr.py#L96):
  Preconditioning is another way to reduce the number of iterations. If it is possible to solve a related system ``M*x = b`` efficiently, where M approximates A in some helpful way (e.g. M - A has low rank or its elements are small relative to those of A), LSQR may converge more rapidly on the system ``A*M(inverse)*z = b``, after which x can be recovered by solving M\*x = z.
  
If A is symmetric, LSQR should not be used! Alternatives are the symmetric conjugate-gradient method (cg) and/or SYMMLQ.  SYMMLQ is an implementation of symmetric cg that applies to any symmetric A and will converge more rapidly than LSQR.  If A is positive definite, there are other implementations of symmetric cg that require slightly less work per iteration than SYMMLQ (but will take the same number of iterations).

# End